<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto1/Spotify_year_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>